In [1]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images

100% 1.68G/1.68G [00:18<00:00, 92.4MB/s]
100% 1.68G/1.68G [00:18<00:00, 96.4MB/s]


In [ ]:
!unzip deepfake-and-real-images.zip

In [9]:
IMGWIDTH = 256

class MesoNet:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        x = Input(shape=(IMGWIDTH, IMGWIDTH, 3))

        x1 = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

        x2 = Conv2D(8, (5, 5), padding='same', activation='relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

        x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

        x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation='sigmoid')(y)

        model = Model(inputs=x, outputs=y)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train(self, x_train, validation_generator, epochs):
        self.model.fit(x_train, validation_data=validation_generator, epochs=epochs, verbose=1)

    def predict_batch(self, x):
        return self.model.predict(x)

    def predict_single(self, img):
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        return self.model.predict(img)

    def evaluate_accuracy(self, x_test):
        self.model.evaluate(x_test, verbose=1, max_queue_size=20)

    def save_model(self, filepath):
        self.model.save(filepath)

    def load_model(self, filepath):
        self.model = load_model(filepath)

In [7]:
# Define paths
train_path = '/content/Dataset/Train'
valid_path = '/content/Dataset/Validation'
test_path = '/content/Dataset/Test'

# Data generators with augmentation
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMGWIDTH, IMGWIDTH),
    batch_size=64,
    class_mode='binary'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(IMGWIDTH, IMGWIDTH),
    batch_size=64,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMGWIDTH, IMGWIDTH),
    batch_size=64,
    class_mode='binary'
)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [ ]:
# Load or instantiate model
model = MesoNet()
history = model.train(train_generator, valid_generator, epochs=2)

In [12]:
model.save_model("Meso4.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
accuracy = model.evaluate_accuracy(test_generator)
print("Test Accuracy:", accuracy)

In [ ]:
import matplotlib.pyplot as plt

def visualize_data(model, data_generator, num_samples=5):
    sample_images, sample_labels = data_generator.next()
    predictions = model.predict_single(sample_images)

    plt.figure(figsize=(15, 8))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(sample_images[i])
        plt.title(f"Predicted: {predictions[i][0]:.2f}, Actual: {sample_labels[i]}")
        plt.axis('off')
    plt.show()

# Usage example
visualize_data(model, test_generator, num_samples=5)

In [ ]:
img_path = ""
prediction = model.predict_single(img_path)

print(f"Prediction score: {prediction}")
print("REAL") if (prediction > 0.6) else print("FAKE")